In [1]:
# import sys; sys.path.append('.')
from setup import *
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 4)
pd.set_option('display.max_columns', 200)

In [2]:
print('Loading 200k tweets (including metadata) could take a minute or so...')
df = pd.read_csv(os.path.join(DATA_PATH, 'all_tweets.csv'), index_col='id', low_memory=False,
                 quotechar='"', quoting=pd.io.common.csv.QUOTE_NONNUMERIC)
# in iPython Notebook print out df.columns to show that many of them contain dots
# rename the columns to be attribute-name friendly
df.columns = [label.replace('.', '_') for label in df.columns]
print('Done.')


Loading 200k tweets (including metadata) could take a minute or so...
Done.


In [3]:
# compress it using python 3 for future loading with python 3
df.to_csv(os.path.join(DATA_PATH, 'all_tweets.csv.gz'), quotechar='"', quoting=pd.io.common.csv.QUOTE_NONNUMERIC, compression='gzip')
# with gzip.open(os.path.join(DATA_PATH, 'all_tweets.csv.gz'), 'w') as fout:
    

### While you wait you can browse [this map](http://www.slaw.ca/wp-content/uploads/2011/11/map-of-a-tweet-copy.pdf) of the metadata that comes with a tweet (a json string)


In [9]:
print('The raw table shape is {}'.format(df.shape))
nonnull_rows = 330
nonnull_cols = 50
df = df.dropna(axis=1, thresh=nonnull_rows)
print('After dropping columns with fewer than {} nonnull values, the table shape is {}'.format(nonnull_rows, df.shape))
df = df.dropna(axis=0, thresh=nonnull_cols)
print('After dropping rows with fewer than {} nonnull values, the table shape is {}'.format(nonnull_cols, df.shape))


# in ipython notebook, explore and describe the DataFrame columns
print('Of the {} columns, {} are actually DataFrames'.format(len(df.columns), sum([not isinstance(df[col], pd.Series) for col in df.columns])))
# remove dataframes with only 2 columns and one is the _str of the other:
for col in df.columns:
    if isinstance(df[col], pd.DataFrame):
        print('Column {} is a {}-wide DataFrame'.format(col, len(df[col].columns)))
        if df[col].columns[1] == df[col].columns[0] + '_str':
            print('Column {} looks easy because it has sub-columns {}'.format(col, df[col].columns))
            df[col] = df[col][df[col].columns[1]]
        else:
            try:
                assert(float(df[col].iloc[:, 0].max()) == float(df[col].iloc[:, 1].max()))
                df[col] = df[col].fillna(-1, inplace=False)
                series = pd.Series([int(Decimal(x)) for x in df[col].iloc[:, 1].values]).astype('int64').copy()
                del df[col]
                df[col] = series
                print('Finished converting column {} to type {}({})'.format(col, type(df[col]), df[col].dtype))
            except:
                print_exc()

print('Of the {} columns, {} are still DataFrames after trying to convert both columns to long integers'.format(
    len(df.columns), sum([not isinstance(df[col], pd.Series) for col in df.columns])))

The raw table shape is (200168, 285)
After dropping columns with fewer than 330 nonnull values, the table shape is (200168, 285)
After dropping rows with fewer than 50 nonnull values, the table shape is (193378, 285)
Of the 285 columns, 8 are actually DataFrames
Column quoted_status_id is a 2-wide DataFrame
Finished converting column quoted_status_id to type <class 'pandas.core.series.Series'>(float64)
Column quoted_status_id_str is a 2-wide DataFrame
Finished converting column quoted_status_id_str to type <class 'pandas.core.series.Series'>(float64)
Column retweeted_status_quoted_status_id is a 2-wide DataFrame
Finished converting column retweeted_status_quoted_status_id to type <class 'pandas.core.series.Series'>(float64)
Column retweeted_status_quoted_status_id_str is a 2-wide DataFrame
Finished converting column retweeted_status_quoted_status_id_str to type <class 'pandas.core.series.Series'>(float64)
Of the 281 columns, 0 are still DataFrames after trying to convert both columns t

In [10]:
print('df.describe() stats:')
desc = df.describe()
for col, stats in desc.T.iterrows():
    print('')
    print('{} ({})'.format(col, df[col].dtype if isinstance(df[col], pd.Series) else type(df[col])))
    print(stats)

df.describe() stats:


/home/hobs/.virtualenvs/AgileMachineLearning/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)



favorite_count (int64)
count    193378.000000
mean          0.629679
std           6.251319
             ...      
50%           0.000000
75%           0.000000
max        1165.000000
Name: favorite_count, dtype: float64

id_str (int64)
count    1.933780e+05
mean     7.274888e+17
std      3.778481e+15
             ...     
50%      7.271970e+17
75%      7.301318e+17
max      7.345639e+17
Name: id_str, dtype: float64

in_reply_to_status_id (float64)
count    1.116500e+04
mean     7.270313e+17
std      1.485877e+16
             ...     
50%               NaN
75%               NaN
max      7.345608e+17
Name: in_reply_to_status_id, dtype: float64

in_reply_to_status_id_str (float64)
count    1.116500e+04
mean     7.270313e+17
std      1.485877e+16
             ...     
50%               NaN
75%               NaN
max      7.345608e+17
Name: in_reply_to_status_id_str, dtype: float64

in_reply_to_user_id (float64)
count    1.300700e+04
mean     2.245103e+16
std      1.249313e+17
            

In [13]:
df

,coordinates_coordinates,coordinates_type,created_at,entities_hashtags,entities_media,entities_symbols,entities_urls,entities_user_mentions,favorite_count,favorited,geo_coordinates,geo_type,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,lat,lon,metadata_iso_language_code,metadata_result_type,place_bounding_box_coordinates,place_bounding_box_type,place_contained_within,place_country,place_country_code,place_full_name,place_id,place_name,place_place_type,place_url,possibly_sensitive,quoted_status_created_at,quoted_status_entities_hashtags,quoted_status_entities_media,quoted_status_entities_symbols,quoted_status_entities_urls,quoted_status_entities_user_mentions,quoted_status_favorite_count,quoted_status_favorited,quoted_status_is_quote_status,quoted_status_lang,quoted_status_metadata_iso_language_code,quoted_status_metadata_result_type,quoted_status_possibly_sensitive,quoted_status_retweet_count,quoted_status_retweeted,quoted_status_source,quoted_status_text,quoted_status_truncated,quoted_status_user_contributors_enabled,quoted_status_user_created_at,quoted_status_user_default_profile,quoted_status_user_default_profile_image,quoted_status_user_description,quoted_status_user_entities_description_urls,quoted_status_user_entities_url_urls,quoted_status_user_favourites_count,quoted_status_user_followers_count,quoted_status_user_friends_count,quoted_status_user_geo_enabled,quoted_status_user_has_extended_profile,quoted_status_user_id,quoted_status_user_id_str,quoted_status_user_is_translation_enabled,quoted_status_user_is_translator,quoted_status_user_lang,quoted_status_user_listed_count,quoted_status_user_location,quoted_status_user_name,quoted_status_user_profile_background_color,quoted_status_user_profile_background_image_url,quoted_status_user_profile_background_image_url_https,quoted_status_user_profile_background_tile,quoted_status_user_profile_banner_url,quoted_status_user_profile_image_url,quoted_status_user_profile_image_url_https,quoted_status_user_profile_link_color,quoted_status_user_profile_sidebar_border_color,quoted_status_user_profile_sidebar_fill_color,quoted_status_user_profile_text_color,quoted_status_user_profile_use_background_image,quoted_status_user_protected,quoted_status_user_screen_name,quoted_status_user_statuses_count,quoted_status_user_time_zone,quoted_status_user_url,quoted_status_user_utc_offset,quoted_status_user_verified,retweet_count,retweeted,retweeted_status_created_at,retweeted_status_entities_hashtags,retweeted_status_entities_media,retweeted_status_entities_symbols,retweeted_status_entities_urls,retweeted_status_entities_user_mentions,...,retweeted_status_quoted_status_user_profile_sidebar_fill_color,retweeted_status_quoted_status_user_profile_text_color,retweeted_status_quoted_status_user_profile_use_background_image,retweeted_status_quoted_status_user_protected,retweeted_status_quoted_status_user_screen_name,retweeted_status_quoted_status_user_statuses_count,retweeted_status_quoted_status_user_time_zone,retweeted_status_quoted_status_user_url,retweeted_status_quoted_status_user_utc_offset,retweeted_status_quoted_status_user_verified,retweeted_status_retweet_count,retweeted_status_retweeted,retweeted_status_source,retweeted_status_text,retweeted_status_truncated,retweeted_status_user_contributors_enabled,retweeted_status_user_created_at,retweeted_status_user_default_profile,retweeted_status_user_default_profile_image,retweeted_status_user_description,retweeted_status_user_entities_description_urls,retweeted_status_user_entities_url_urls,retweeted_status_user_favourites_count,retweeted_status_user_followers_count,retweeted_status_user_friends_count,retweeted_status_user_geo_enabled,retweeted_status_user_has_extended_profile,retweeted_status_user_id,retweeted_status_user_id_str,retweeted_status_user_is_translation_enabled,retweeted_status_user_is_translator,retweeted_status_user_lang,retweeted_status_

In [18]:
stats = df.describe()
columns = [c for c in stats.columns if stats[c]['count'] > 10000 or 'fav' in c or 'retweet' in c or df[c].dtype in (int, float, np.float64)] + ['text', 'favorite_count', 'geo_coordinates']
print(df.shape)
print(df[columns].shape)
for c in columns:
    print(c)
df.text

/home/hobs/.virtualenvs/AgileMachineLearning/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


(193378, 281)
(193378, 60)
favorite_count
id_str
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
lat
lon
quoted_status_favorite_count
quoted_status_retweet_count
quoted_status_user_favourites_count
quoted_status_user_followers_count
quoted_status_user_friends_count
quoted_status_user_id
quoted_status_user_id_str
quoted_status_user_listed_count
quoted_status_user_statuses_count
quoted_status_user_utc_offset
retweet_count
retweeted_status_favorite_count
retweeted_status_id
retweeted_status_id_str
retweeted_status_in_reply_to_status_id
retweeted_status_in_reply_to_status_id_str
retweeted_status_in_reply_to_user_id
retweeted_status_in_reply_to_user_id_str
retweeted_status_quoted_status_favorite_count
retweeted_status_quoted_status_retweet_count
retweeted_status_quoted_status_user_favourites_count
retweeted_status_quoted_status_user_followers_count
retweeted_status_quoted_status_user_friends_count
retweeted_status_quoted_status_user_id
retweeted_s

id
731122251278499841    RT @javacodegeeks: Top Performance Metrics for...
724281574129180672    World's Largest Python Discovered in Nepal: WA...
                                            ...                        
724275578879111169    Watching Boa vs. Python — https://t.co/5THbrirfQO
724275568871673857    Чертова дюжина вакансий в IT и Digital /  / 1....
Name: text, dtype: object

In [19]:
df[columns].to_csv(os.path.join(DATA_PATH, 'cleaned_tweets.csv.gz'), compression='gzip', encoding='UTF-8', quotechar='"', quoting=pd.io.common.csv.QUOTE_NONNUMERIC)

In [20]:
rawlen = len(df)
df.drop_duplicates('id_str', keep='last', inplace=True)
rawlen - len(df)

10308

In [21]:
df[columns].to_csv(os.path.join(DATA_PATH, 'deduped_tweets.csv.gz'), compression='gzip', encoding='UTF-8', quotechar='"', quoting=pd.io.common.csv.QUOTE_NONNUMERIC)